In [1]:
from pygraphblas import *
from itertools import repeat
from time import time
import random
from numba import njit
from pygraphblas.demo.gviz import draw

options_set(nthreads=1)

In [2]:
from collections import defaultdict
from itertools import groupby
from operator import itemgetter

def group_labels(T):
    d = defaultdict(list)
    for k,v in groupby(T, itemgetter(1)):
        d[k].append(list(v)[0][0])
    return d

def compare_groups(left, right):
    left = {k: set(v) for k, v in left.items()}
    right = {k: set(v) for k, v in right.items()}
    return sorted(left.values()) == sorted(right.values())

def get_louvain_cluster_assignments(cluster_matrix):
    clusters = Vector.sparse(UINT64, cluster_matrix.nrows)
    index_of_vec = Vector.sparse(UINT64, cluster_matrix.ncols)
    for i in range(cluster_matrix.nrows):
        index_of_vec[i] = i
    cluster_matrix.mxv(index_of_vec, out=clusters, semiring=UINT64.MAX_SECOND)
    return clusters

In [3]:
M = Matrix.from_lists(
    [0, 0, 0, 0,
    1, 1, 1, 1,
    2, 2, 2, 2,
    3, 3, 3, 3,
    4, 4, 4, 4,
    5, 5, 5, 5, 5,
    6, 6, 6,
    7, 7, 7, 7],
    [0, 2, 3, 6,
    1, 2, 3, 7,
    0, 2, 4, 6,
    0, 1, 3, 5,
    0, 2, 4, 6,
    1, 3, 5, 6, 7,
    0, 4, 6,
    1, 3, 5, 7],
    list(repeat(1.0, 32)),
    typ=FP64,
)

@njit
def seed_rng(seed):
    random.seed(seed)

@unary_op(FP64)  
def random_scaler(x): 
    return random.uniform(0.0001, x)

AttributeError: module 'pygraphblas.types' has no attribute 'double'

In [ ]:
def louvain_cluster(graph, max_iters=20, seed=None):
    if seed:
        seed_rng(seed)
    assert graph.square, 'First input matrix must be square.'
    start = time()
    rows = graph.nrows
    ApAT = graph.transpose(, accum=FP64.PLUS)
    k = graph.reduce_vector()
    m = k.reduce_int() / 2.0    
    S = Matrix.identity(BOOL, rows, format=lib.GxB_BY_COL)
    S_row = Vector.sparse(BOOL, rows)
    empty = Vector.sparse(BOOL, rows)   
    v = Vector.sparse(FP64, rows)
    vertices_changed = True
    iters = 0
    
        while vertices_changed and iters < max_iters:
        vertices_changed = False
        for i in range(rows):
            if i in k:
                S.extract_col(i, out=S_row)
                S.assign_row(i, empty)
                ApAT.extract_row(i, out=v)
                w = k.dup()
                w.assign_scalar(-k[i]/m, accum=FP64.TIMES, mask=k)
                v += w
                q = v @ S
                kappa = q.reduce_float(FP64.MAX_MONOID)
                t = q.select('==', kappa)
                while len(t) != 1:
                    p = t.apply(random_scaler)
                    max_p = p.reduce_float(FP64.MAX_MONOID)
                    t = p.select('==', max_p)
                S.assign_row(i, t)
                if not t.iseq(S_row):
                    vertices_changed = True
        iters += 1
    print('Input size ', len(graph), ' took ', time() - start)
    return S

In [ ]:
ans = louvain_cluster(M, 10)
labels = get_louvain_cluster_assignments(ans)
draw(M, show_weight=False, label_vector=labels)

In [ ]:
sizes = [(6, 50), (4, 100), (3, 500), (3, 1000), (3, 5000)]
#sizes = [(3, 5000)]

for iters, size in sizes:
    with open('louvain_data/simulated_blockmodel_graph_%s_nodes.tsv' % size) as challenge:
        data = Matrix.from_tsv(challenge, FP64, size, size) #, format=lib.GxB_BY_ROW)
        print('Loaded', size, 'nodes.')
        ans = louvain_cluster(data, max_iters=iters, seed=42)
        assignments = get_louvain_cluster_assignments(ans)
    with open('louvain_data/simulated_blockmodel_graph_%s_nodes_truePartition.tsv' % size) as truth:
        T = Vector.sparse(UINT8, size)
        for line in truth:
            i, j = map(int, line.split('\t'))
            T[i-1] = j
    print(size, 'Correct?', compare_groups(group_labels(assignments), group_labels(T)))
    

In [ ]:
import pdb; pdb.pm()


> /home/jovyan/pygraphblas/unaryop.py(28)__init__()
     26         self.token = None
     27         self.__class__._auto_unaryops[name][types.Type.gb_from_name(typ)] = op
---> 28         cls = getattr(types, typ)
     29         setattr(cls, name, self)
     30 

ipdb> p op
<cdata 'struct GB_UnaryOp_opaque *' 0x556618d0a910>
ipdb> p typ
'double'
ipdb> list
     23     def __init__(self, name, typ, op):
     24         self.name = "_".join((name, typ))
     25         self.unaryop = op
     26         self.token = None
     27         self.__class__._auto_unaryops[name][types.Type.gb_from_name(typ)] = op
---> 28         cls = getattr(types, typ)
     29         setattr(cls, name, self)
     30 
     31     def __enter__(self):
     32         self.token = current_uop.set(self)
     33         return self

ipdb> up
> /home/jovyan/pygraphblas/unaryop.py(121)inner()
    119         )
    120 
--> 121         return UnaryOp(func_name, arg_type.C, out[0])
    122 
    123     return inner
